In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
#Wrangling----------------------------------------------------------------------------------------------------------------------

In [4]:
#1Merge datasets

# Load the datasets
customer_profiles = pd.read_csv("customer_profiles.csv")
ad_events = pd.read_csv("ad_events.csv")

# Merge the datasets on the 'id' and 'customer' columns using left join
merged_data = pd.merge(customer_profiles, ad_events, left_on='id', right_on='customer', how='left')

# For non-matching rows, fill in NaN values with the corresponding 'id' or 'customer' value
merged_data['customer'].fillna(merged_data['id'], inplace=True)
merged_data['id'].fillna(merged_data['customer'], inplace=True)


dfO=merged_data


In [5]:
#2 Split data into testing and training
#df0=dfO
#--------------------------------------------------------------------------------------------------------------------------


# Assuming dfO is your original dataframe
train_df, test_df = train_test_split(dfO, test_size=0.3, random_state=42)

# train_df will contain 70% of the data
# test_df will contain 30% of the data

In [6]:
#len(train_df)

In [7]:
#split numerical parts of dataframe
df = pd.DataFrame((((train_df.drop('id', axis=1)).drop('home_state', axis=1)).drop('event', axis=1)).drop('event_info', axis=1))

# Make float dataset
df1 = {}

# Extract year, month, and day from "became_member_on (year month day)"
df1["year"] = df["became_member_on (year month day)"].apply(lambda x: float(str(x)[:4]) if pd.notnull(x) and len(str(x)) == 10 else 0)
df1["month"] = df["became_member_on (year month day)"].apply(lambda x: float(str(x)[4:6]) if pd.notnull(x) and len(str(x)) == 10 else 0)
df1["day"] = df["became_member_on (year month day)"].apply(lambda x: float(str(x)[6:8]) if pd.notnull(x) and len(str(x)) == 10 else 0)

# Other columns
df1["gender"] = df["gender"].map({'F': 0.0, 'M': 1.0}).fillna(0.5790270890798996)
df1["ever_married"] = df["ever_married"].map({'No': 0.0, 'Yes': 1.0}).fillna(0.5247599943435656)
df1["time"] = df["time"].apply(lambda x: float(x) if pd.notnull(x) else 366.6747497813369)
df1["age"] = df["age"].apply(lambda x: float(x) if pd.notnull(x) else 53.81169310700612)
df1["income"] = df["income"].apply(lambda x: float(x) if pd.notnull(x) else 64304.72993144192)
df1["kids"] = df["kids"].apply(lambda x: float(x) if pd.notnull(x) else 1.692206166561045)

# Create a new DataFrame
df = (pd.DataFrame(df1))
print((df["gender"]).mean())

0.5790270890798996


In [ ]:
#choose what things too look at


In [8]:
#df.info()
df.head()

,year,month,day,gender,ever_married,time,age,income,kids
28019,0,0,0,0.0,0.0,336.0,81.0,52000.0,0.0
106711,0,0,0,1.0,0.0,600.0,52.0,97000.0,2.0
218556,0,0,0,0.0,1.0,108.0,77.0,92000.0,3.0
272639,0,0,0,0.0,0.0,60.0,70.0,39000.0,0.0
42512,0,0,0,1.0,0.0,336.0,51.0,57000.0,0.0


In [ ]:
#KMeans--------------------------------------------------------------------------------------------------------------------

In [ ]:
#1get data
#df = pd.read_csv("demo.csv")
df.describe()

In [ ]:
# Import stuff needed
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.cluster import KMeans

In [ ]:
#Scale down data
scaled_df = pd.DataFrame(scaler.fit_transform(df))

In [ ]:
#make inertias stuff
inertias = []
for cluster in range(1,20):
    kmeans = KMeans(n_clusters=cluster)
    kmeans.fit(scaled_df)
    inertias.append(kmeans.inertia_)

In [ ]:
#Make dataframe from different inertias
frame = pd.DataFrame({"Clusters":range(1,20), "Inertias": inertias})

In [ ]:
#3 View it

import seaborn as sns
sns.lineplot(data=frame, x="Clusters", y="Inertias")

In [ ]:
#
kmeans = KMeans(n_clusters=6)
kmeans.fit(scaled_df)
predictions = kmeans.predict(scaled_df)

#get the centers
centers = kmeans.cluster_centers_
unscaled_centers = scaler.inverse_transform(centers)